## Szükséges csomagok

In [1]:
import os
import json
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import SiglipProcessor, SiglipModel
from sklearn.metrics import average_precision_score
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\Adam\miniconda3\envs\main_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Modell és az adatok betöltése

In [ ]:
# Betöltés
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SiglipModel.from_pretrained("google/siglip-base-patch16-224").to(device)
processor = SiglipProcessor.from_pretrained("google/siglip-base-patch16-224")

# Adatok
embeddings = torch.load("../data/image_embeddings_siglip.pt")
infos = torch.load("../data/image_info_siglip.pt")
with open("../data/custom_captions.json", encoding="utf-8") as f:
    class_texts = json.load(f)

C:\Users\Adam\AppData\Local\Temp\ipykernel_5460\1434608005.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load("C:/Users/Adam/Desktop/applied_ml/data

## Képek csoportosítása osztályonként

In [ ]:
# Ez egy szótár, amely minden osztályhoz hozzárendeli a hozzá tartozó képek embeddingjeit
class_to_embeddings = {}

for idx, info in enumerate(infos):
    label = str(info['label'])
    if label not in class_to_embeddings:
        class_to_embeddings[label] = []
    class_to_embeddings[label].append(embeddings[idx])

## Osztályon belüli hasonlóságok számítása

Minden osztály esetén kiszámoljuk az osztályon belüli képpárok átlagos cosine similarity értékét, majd elmentjük JSON fájlba.


In [ ]:
# Kiszámoljuk az osztályok közötti átlagos cosine similarity-t
similarities_per_class = {}

for class_id, class_embeddings in class_to_embeddings.items():
    embeddings_tensor = torch.stack(class_embeddings)
    
    # Kiszámítjuk a cosine similarity mátrixot az osztályon belül
    similarity_matrix = cosine_similarity(embeddings_tensor.cpu().numpy())
    
    # Az összes párra átlagos cosine similarity-t számolunk
    upper_triangle_indices = np.triu_indices(similarity_matrix.shape[0], k=1)
    
    avg_similarity = similarity_matrix[upper_triangle_indices].mean()
    similarities_per_class[class_id] = avg_similarity.item()  # float32 -> float64

# Eredmény mentése fájlba
output_file = "../data/intra_class_similarities.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(similarities_per_class, f, indent=4)

## Osztályok közötti hasonlóságok számítása

Két osztály minden képpárjára kiszámoljuk a cosine similarity értékét. Az azonos osztályhoz tartozó képek esetén kizárjuk az önmagukkal való összehasonlítást.

In [ ]:
# Képosztályok közötti inter cosine similarity kiszámítása
inter_class_similarities = {}

class_ids = list(class_to_embeddings.keys())

for class_id_1 in class_ids:
    embeddings_class_1 = torch.stack(class_to_embeddings[class_id_1]).cpu().numpy()
    inter_class_similarities[class_id_1] = {}

    for class_id_2 in class_ids:
        embeddings_class_2 = torch.stack(class_to_embeddings[class_id_2]).cpu().numpy()

        if class_id_1 == class_id_2:
            # Osztályon belül: saját magukkal való hasonlóságok kizárása (diagonálon kívüli elemek)
            similarity_matrix = cosine_similarity(embeddings_class_1)
            if similarity_matrix.shape[0] > 1:
                upper_triangle_indices = np.triu_indices(similarity_matrix.shape[0], k=1)
                avg_similarity = similarity_matrix[upper_triangle_indices].mean()
            else:
                avg_similarity = 1.0  # Ha csak 1 kép van az osztályban, nincs mit számolni
        else:
            # Két különböző osztály: minden páron számolunk
            similarity_matrix = cosine_similarity(embeddings_class_1, embeddings_class_2)
            avg_similarity = similarity_matrix.mean()

        inter_class_similarities[class_id_1][class_id_2] = float(avg_similarity)

# Eredmény mentése fájlba
output_file = "../data/inter_class_similarities.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(inter_class_similarities, f, indent=4)

## Osztályleírások és képek hasonlóságának mérése

Minden osztályhoz tartozó szöveg alapján kiszámoljuk a legjobban egyező kép cosine similarity értékét az embeddingek között.  
Ezt ha ezen futtatjuk: `short_custom_captions.json` látható, hogy egy rövid mondat semmiképp sem lesz elég.

In [ ]:
# Eredmények tárolása
top1_results = []

# Végigmegyünk mind a 102 osztályon
for class_id in tqdm(range(1, 103), desc="Top-1 similarity számolása"):
    str_id = str(class_id)
    lines = class_texts[str_id]
    full_text = " ".join(lines)

    # Szöveg → embedding
    inputs = processor(text=[full_text], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_feat = model.get_text_features(**inputs)
        text_feat = text_feat / text_feat.norm(p=2, dim=-1, keepdim=True)
        text_feat = text_feat.cpu()

    # Cosine similarity az összes képre
    similarities = torch.matmul(embeddings, text_feat.T).squeeze()

    # Top-1 érték kinyerése
    top1_score = similarities.max().item()

    top1_results.append({
        "class_id": class_id,
        "top1_similarity": round(top1_score, 5)
    })

# DataFrame-be tesszük
df_top1 = pd.DataFrame(top1_results)
print(df_top1.head())

# CSV-be mentés (ha szeretnéd)
df_top1.to_csv("../data/class_top1_similarity.csv", index=False)

Top-1 similarity számolása: 100%|██████████| 102/102 [00:01<00:00, 94.35it/s] 

   class_id  top1_similarity
0         1          0.02080
1         2          0.02340
2         3          0.00329
3         4          0.02044
4         5          0.00179


## Precision@N számítása osztályonként

Minden osztály leírása alapján kiszámoljuk, hogy az embeddingek közül a hozzá tartozó képek közül hány jelenik meg a legjobban illeszkedők között. Az N értéke az adott osztályhoz tartozó képek száma.

In [ ]:
# Eredmények tárolása
precision_results = []

# Végigmegyünk mind a 102 osztályon
for class_id in tqdm(range(1, 103), desc="Precision@N számolása"):
    str_id = str(class_id)
    lines = class_texts[str_id]
    full_text = " ".join(lines)

    # Szöveg → embedding
    inputs = processor(text=[full_text], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_feat = model.get_text_features(**inputs)
        text_feat = text_feat / text_feat.norm(p=2, dim=-1, keepdim=True)
        text_feat = text_feat.cpu()

    # Cosine similarity az összes képre
    similarities = torch.matmul(embeddings, text_feat.T).squeeze()

    # Az adott osztályhoz tartozó képek száma
    class_size = sum(1 for info in infos if int(info["label"]) == class_id)

    # Top N visszakeresés (N = class_size)
    top_indices = similarities.topk(class_size).indices

    # Ezzel nézzük meg hány top kép van a class_id osztályban
    correct = sum(1 for idx in top_indices if int(infos[idx]["label"]) == class_id)

    precision = correct / class_size if class_size > 0 else 0.0

    precision_results.append({
        "class_id": class_id,
        "class_size": class_size,
        "correct_in_topN": correct,
        "precision_at_class_size": round(precision, 5)
    })

# DataFrame-be tesszük
df_precision = pd.DataFrame(precision_results)
print(df_precision.head())

# CSV mentés
df_precision.to_csv("../data/class_precision_at_class_size.csv", index=False)

Precision@N számolása: 100%|██████████| 102/102 [00:01<00:00, 72.92it/s]

   class_id  class_size  correct_in_topN  precision_at_class_size
0         1          27               20                  0.74074
1         2          49               40                  0.81633
2         3          36               25                  0.69444
3         4          44               39                  0.88636
4         5          54               39                  0.72222


## Recall@X% küszöbértékek számítása

Minden osztály esetén meghatározzuk, hogy hány "legjobb" kép szükséges ahhoz, hogy az osztályhoz tartozó képek 70%, 80%, 90% és 95%-át visszakeressük. Emellett elmentjük a megfelelő cosine similarity küszöbértékeket is, amelyek ezeket az arányokat biztosítják.

In [ ]:
# Kimenet tárolása
recall_results = []

# Osztályonként végigmegyünk
for class_id in tqdm(range(1, 103), desc="Recall cutoff számolása"):
    str_id = str(class_id)
    lines = class_texts[str_id]
    full_text = " ".join(lines)

    # Text embedding
    inputs = processor(text=[full_text], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_feat = model.get_text_features(**inputs)
        text_feat = text_feat / text_feat.norm(p=2, dim=-1, keepdim=True)
        text_feat = text_feat.cpu()

    # Cosine similarity az összes képre
    similarities = torch.matmul(embeddings, text_feat.T).squeeze()
    sorted_indices = torch.argsort(similarities, descending=True)

    # Bináris vektor: 1, ha helyes osztályba tartozik
    binary_hits = [1 if int(infos[i]["label"]) == class_id else 0 for i in sorted_indices]
    cumsum_hits = np.cumsum(binary_hits) # a visszakeresett listában hány jó találat van eddig összesítve

    class_size = sum(binary_hits)  # az adott osztályhoz tartozó képek száma
    recall_targets = {
        70: int(np.ceil(class_size * 0.70)),
        80: int(np.ceil(class_size * 0.80)),
        90: int(np.ceil(class_size * 0.90)),
        95: int(np.ceil(class_size * 0.95)),
    }

    topN_at = {}
    threshold_at = {}

    for perc, target in recall_targets.items():
        found_index = next((i for i, val in enumerate(cumsum_hits) if val >= target), None) # keressük azt az első olyan indexet, ahol már elértük vagy túlléptük a kívánt számú helyes találatot
        if found_index is not None:
            topN_at[f"topN_{perc}"] = found_index + 1  # index → N
            threshold_at[f"threshold_{perc}"] = round(similarities[sorted_indices[found_index]].item(), 5)
        else:
            topN_at[f"topN_{perc}"] = None
            threshold_at[f"threshold_{perc}"] = None

    result = {
        "class_id": class_id,
        "class_size": class_size,
        **topN_at,
        **threshold_at
    }

    recall_results.append(result)

# Eredmények táblázatba
df_recall = pd.DataFrame(recall_results)
print(df_recall.head())

# Mentés CSV-be
df_recall.to_csv("../data/class_recall_cutoffs.csv", index=False)

C:\Users\Adam\AppData\Local\Temp\ipykernel_17880\944431738.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load("C:/Users/Adam/Desktop/applied_ml/dat

   class_id  class_size  topN_70  topN_80  topN_90  topN_95  threshold_70  \
0         1          27       26       34       50       51       0.13915   
1         2          49       37       45       75       86       0.09562   
2         3          36       39       47       67      227       0.11541   
3         4          44       32       39       45       52       0.12268   
4         5          54       50       80       90      114       0.11872   

   threshold_80  threshold_90  threshold_95  
0       0.13033       0.11447       0.11366  
1       0.08807       0.07893       0.07664  
2       0.11272       0.10759       0.08375  
3       0.11493       0.10914       0.10568  
4       0.11281       0.11144       0.10730  


## Average Precision számítása

Minden osztály esetén kiszámoljuk az osztályhoz tartozó képek visszakeresésének average precision értékét a cosine similarity alapján. A végén kiszámoljuk a teljes rendszerre vonatkozó mAP értéket is.

In [ ]:
# Eredmények
ap_results = []

for class_id in tqdm(range(1, 103), desc="AP számolása"):
    str_id = str(class_id)
    lines = class_texts[str_id]
    full_text = " ".join(lines)

    # Szöveg embedding
    inputs = processor(text=[full_text], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_feat = model.get_text_features(**inputs)
        text_feat = text_feat / text_feat.norm(p=2, dim=-1, keepdim=True)
        text_feat = text_feat.cpu()

    # Cosine similarity
    similarities = torch.matmul(embeddings, text_feat.T).squeeze().numpy()

    # Binary ground truth: 1 ha jó osztály, különben 0
    gt_labels = [1 if int(info["label"]) == class_id else 0 for info in infos]

    # Average precision
    if sum(gt_labels) > 0:
        ap = average_precision_score(gt_labels, similarities)
    else:
        ap = 0.0

    ap_results.append({
        "class_id": class_id,
        "average_precision": round(ap, 5)
    })

# DataFrame + mAP
df_ap = pd.DataFrame(ap_results)
mean_ap = df_ap["average_precision"].mean()
print(df_ap.head())
print(f"\nMean Average Precision (mAP): {round(mean_ap, 5)}")

# CSV mentés
df_ap.to_csv("../data/class_average_precision.csv", index=False)

C:\Users\Adam\AppData\Local\Temp\ipykernel_17880\4052214356.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load("C:/Users/Adam/Desktop/applied_ml/da

   class_id  average_precision
0         1            0.79286
1         2            0.91528
2         3            0.78243
3         4            0.96496
4         5            0.81550

Mean Average Precision (mAP): 0.87911


## Kép-alapú top-1 hasonlóság számítása

Kiszámoljuk az összes kép cosine similarity mátrixát, majd minden képhez meghatározzuk a legjobban hasonlító másik képet (önmagát kizárva). Megnézzük, hogy ez a top-1 kép azonos osztályba tartozik-e?


In [ ]:
# Cosine similarity mátrix (n x n)
embedding_matrix = embeddings.numpy()
sim_matrix = cosine_similarity(embedding_matrix)

# Diagonálisan 1-es lenne (önmagával) – ezt kilőjük
np.fill_diagonal(sim_matrix, -1)

# Eredmények tárolása
top1_results = []

for i in tqdm(range(sim_matrix.shape[0]), desc="Top-1 keresés képenként"):
    top1_idx = sim_matrix[i].argmax()
    top1_sim = sim_matrix[i][top1_idx]

    label_i = int(infos[i]["label"])
    label_top1 = int(infos[top1_idx]["label"])
    same_class = (label_i == label_top1)

    top1_results.append({
        "image_index": i,
        "top1_index": top1_idx,
        "top1_similarity": round(top1_sim, 5),
        "label": label_i,
        "top1_label": label_top1,
        "same_class": same_class
    })

# Mentés
df_top1 = pd.DataFrame(top1_results)
df_top1.to_csv("../data/image_top1_similarity.csv", index=False)

Top-1 keresés képenként: 100%|██████████| 6552/6552 [00:00<00:00, 148134.00it/s]


## Kép-alapú Recall@K számítása

Minden kép esetén meghatározzuk, hogy a K legjobban hasonlító képből hány tartozik azonos osztályba. Ezt különböző K értékekre (1, 5, 10, 20) számoljuk ki.  
Itt azért olyan kicsik az értékek mindig, mert az i-edik kép osztályának létszámával osztunk (-1, mert i nincs benne). 

In [ ]:
# Cosine similarity mátrix (n x n)
embedding_matrix = embeddings.numpy()
sim_matrix = cosine_similarity(embedding_matrix)

# Diagonálisan -1, hogy önmagát ne hozza vissza
np.fill_diagonal(sim_matrix, -1)

# Recall@K értékek
K_values = [1, 5, 10, 20]
recall_records = []

for i in tqdm(range(sim_matrix.shape[0]), desc="Recall képenként"):
    true_label = int(infos[i]["label"])
    top_indices = np.argsort(-sim_matrix[i])  # csökkenő sorrend

    recalls = {}
    # Az aktuális osztályhoz tartozó képek száma (önmagát kivéve)
    total_same_class = sum(1 for j in range(len(infos)) if j != i and int(infos[j]["label"]) == true_label)

    for k in K_values:
        top_k = top_indices[:k]
        correct = sum(1 for j in top_k if int(infos[j]["label"]) == true_label)
        recall = correct / total_same_class if total_same_class > 0 else 0.0
        recalls[f"recall@{k}"] = round(recall, 5)

    recall_records.append({
        "image_index": i,
        "label": true_label,
        **recalls
    })

# DataFrame mentés
df_recall = pd.DataFrame(recall_records)
df_recall.to_csv("../data/image_recall_at_k.csv", index=False)

# Átlag recall@k kiírás
mean_recalls = df_recall[[f"recall@{k}" for k in K_values]].mean()
print("\nÁtlagos Recall@K értékek:")
for k in K_values:
    print(f"Recall@{k}: {round(mean_recalls[f'recall@{k}'], 5)}")

Recall képenként: 100%|██████████| 6552/6552 [00:06<00:00, 1051.37it/s]


Átlagos Recall@K értékek:
Recall@1: 0.01568
Recall@5: 0.07727
Recall@10: 0.15266
Recall@20: 0.29798


## Kép-alapú Average Precision számítása

Minden kép esetén kiszámoljuk, hogy a hozzá hasonlító többi kép mennyire jól sorolja előre az azonos osztályúakat a cosine similarity alapján. Az önmagával való hasonlóságot kizárjuk.

In [ ]:
# Cosine similarity mátrix
embedding_matrix = embeddings.numpy()
sim_matrix = cosine_similarity(embedding_matrix)

# Saját magát nem hasonlítjuk (AP-ben nincs értelme)
np.fill_diagonal(sim_matrix, -1)

# Eredmények
ap_results = []

for i in tqdm(range(sim_matrix.shape[0]), desc="AP per image"):
    label = int(infos[i]["label"])

    # Ground truth: 1 ha ugyanabba az osztályba tartozik, különben 0 (önmagát nem számítjuk)
    gt = np.array([1 if int(infos[j]["label"]) == label else 0 for j in range(len(infos))])
    gt[i] = 0  # önmagát nullázzuk

    scores = sim_matrix[i]

    if gt.sum() > 0:
        ap = average_precision_score(gt, scores)
    else:
        ap = 0.0

    ap_results.append({
        "image_index": i,
        "label": label,
        "average_precision": round(ap, 5)
    })

# DataFrame + mAP
df_ap = pd.DataFrame(ap_results)
mean_ap = df_ap["average_precision"].mean()
print(df_ap.head())
print(f"\nMean Average Precision (mAP) for images: {round(mean_ap, 5)}")

# Mentés
df_ap.to_csv("../data/image_average_precision.csv", index=False)

AP per image: 100%|██████████| 6552/6552 [00:18<00:00, 352.19it/s]

   image_index  label  average_precision
0            0      1            0.80528
1            1      1            0.80784
2            2      1            0.80397
3            3      1            0.79885
4            4      1            0.77852

Mean Average Precision (mAP) for images: 0.8723
